In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random

from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [2]:
from utils import generateClickEvent, generateImpressionEvent

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [3]:
generateEvents(False, False)

{"impressionId": "b5977d2d-8188-453d-9267-a094d4b1bb55", "impressionDate": "2024-01-04T21:29:21", "productId": 792, "browser": "Safari", "browserVersion": "8.5", "device": "computer", "source": "organic", "ip_address": "67.100.108.144", "landing_page": "https://www.collins-thompson.com/tag/taghome.asp", "page_loading_seconds": 5.979}
{"clickId": "3d40b3d9-5564-4eed-ac00-4414f5726671", "impressionId": "b5977d2d-8188-453d-9267-a094d4b1bb55", "clickDate": "2024-01-04T21:29:21", "productId": 792}
{"impressionId": "6872b98a-bb2a-443b-81d5-cb940b27b737", "impressionDate": "2024-01-07T02:38:38", "productId": 880, "browser": "Firefox", "browserVersion": "13.6", "device": "tablet", "source": "google", "ip_address": "26.209.224.172", "landing_page": "https://ross.info/tag/searchfaq.asp", "page_loading_seconds": 2.529}
{"impressionId": "69218a30-252a-4c28-9c6d-4ed0bde7f5a0", "impressionDate": "2024-01-06T23:31:19", "productId": 926, "browser": "Safari", "browserVersion": "8.5", "device": "mobile"